In [17]:
suppressMessages({library(Seurat)
                  library(Matrix)
                  library(dplyr)
                  library(ggplot2)
                  library(data.table)
                 library(stringr)
                 library(tibble)})
#library(Seurat)
#library(Matrix)
#library(dplyr)
#library(ggplot2)
#library(data.table)

In [2]:
setwd('/data/brian/courses/singlecell')


In [6]:
samples <- as.list(list.files(path = 'covid_balf/singlecellstudy/data', pattern='*.h5', recursive= FALSE))
moderate <- samples[c(1,2,4)]
severe <- samples[c(3,5,6,10,11,12)]
healthy <- samples[seq(7,9,1)]

In [7]:
Get_paths<- function(listing){
    datadir<- list()
    for (i in 1:length(listing)){
        datadir[i]<- paste('/data/brian/courses/singlecell/covid_balf/',listing[i], sep="")}
    return(datadir)
}

Get_patient<- function(pat){
    patients <- list()
    for (i in pat){
        patients[i] <- strsplit(i[[1]], "[_]")[[1]][3]
    }
    return (patients)
}

Create_Seurat<- function(datas){
    datas<- Get_paths(datas)
    ndata<- data.table(ID = levels(as.factor(unlist(datas))))
    patients <- Get_patient(datas)
    for (i in datas){
            df.data <- data.frame()
            df <- data.frame()
            ndata$data <- sapply(ndata$ID, function(i, df.data, df){
                df.data <- Seurat::Read10X_h5(i ,use.names = TRUE, unique.features = TRUE)
                df <- CreateSeuratObject(counts = df.data , project = unlist(patients[i]), 
                                         min.cells = 3, min.features = 200)})
        for (j in 1:dim(ndata)[1]){
            ndata$data[[j]][['percent.mito']] <- PercentageFeatureSet(ndata$data[[j]], pattern = "^MT-")
        }
        return (ndata)
        }
}

In [8]:
moderate.df <- Create_Seurat(moderate)

Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' has been deprecated; setting 'repr = "T"' for you”
Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' has been deprecated; setting 'repr = "T"' for you”
Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' has been deprecated; setting 'repr = "T"' for you”


In [9]:
severe.df<- Create_Seurat(severe)

Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' has been deprecated; setting 'repr = "T"' for you”
Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' has been deprecated; setting 'repr = "T"' for you”
Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' has been deprecated; setting 'repr = "T"' for you”
Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' has been deprecated; setting 'repr = "T"' for you”
Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' has been deprecated; setting 'repr = "T"' for you”
Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' has been deprecated; setting 'repr = "T"' for you”


In [10]:
healthy.df<- Create_Seurat(healthy) # I did not include GSM3660650

Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' has been deprecated; setting 'repr = "T"' for you”
Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' has been deprecated; setting 'repr = "T"' for you”
Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' has been deprecated; setting 'repr = "T"' for you”


In [19]:
dpi <- 300
Preprocess_vis <- function(df){
    for (i in df[['data']]){
        print(dim(i))
        png(file=paste(as.character(unique(unlist(i@meta.data$orig.ident))),"_qc.png",sep=''), width = dpi*16, height = dpi*8, units = "px",res = dpi,type='cairo')
        print(VlnPlot(object = i, features = c("nFeature_RNA", "nCount_RNA", "percent.mito"), ncol = 3))
        dev.off()
        png(file=paste(as.character(unique(unlist(i@meta.data$orig.ident))),"_umi-mito.png",sep=''), width = dpi*6, height = dpi*5, units = "px",res = dpi,type='cairo')
        print(FeatureScatter(object = i, feature1 = "nCount_RNA", feature2 = "percent.mito"))
        dev.off()
        png(paste(as.character(unique(unlist(i@meta.data$orig.ident))),"_umi-gene.png",sep=''), width = dpi*6, height = dpi*5, units = "px",res = dpi,type='cairo')
        print(FeatureScatter(object = i, feature1 = "nCount_RNA", feature2 = "nFeature_RNA"))
        dev.off()
    }
}

In [20]:
Preprocess_vis(healthy.df)

[1] 16848 11112
[1] 15886 10361
[1] 18477  6583


In [21]:
Preprocess_vis(moderate.df)

[1] 20271  4233
[1] 19966  4562
[1] 15935   915


In [22]:
Preprocess_vis(severe.df)

[1] 21112 20289
[1] 19850 17396
[1] 17947  3401
[1] 19398  2397
[1] 18138  2485
[1] 19340  6962


In [11]:
Preprocess_filter <- function(df){
    nFeature_RNA_low = 200
    nFeature_RNA_high = 6000
    nCount_RNA_req = 1000
    percent.mito.req = 10
    df_list <- list()
    for (i in df[['data']]){
        nCoV.seurat.filter <- subset(x = i, subset = nFeature_RNA > nFeature_RNA_low
                                     & nFeature_RNA < nFeature_RNA_high 
                                     & nCount_RNA > nCount_RNA_req & percent.mito < percent.mito.req)
        nCov.seurat.filter.norm <- NormalizeData(nCoV.seurat.filter, verbose = FALSE)
        nCov.seurat.filter.norm.selection <- FindVariableFeatures(nCov.seurat.filter.norm, selection.method = "vst", 
                                                                  nfeatures = 2000,verbose = FALSE)
        df_list <- append(df_list, nCov.seurat.filter.norm.selection)
    }
    return (df_list)}

In [13]:
healthy.df.filtered <- Preprocess_filter(healthy.df) # I did not include GSM3660650
moderate.df.filtered<- Preprocess_filter(moderate.df)
severe.df.filtered<- Preprocess_filter(severe.df)

In [14]:
healthy.df.filtered[[1]]

An object of class Seurat 
16848 features across 8466 samples within 1 assay 
Active assay: RNA (16848 features, 2000 variable features)

In [15]:
metadata_change <- function(df, string){
    df.metadata <- df@meta.data 
    df.metadata$ID <- rownames(df.metadata)
    df.metadata <- df.metadata %>%
        mutate(ID = str_replace(ID, "-1", string))
    df <- AddMetaData(object = df,
                     metadata = df.metadata)
    rownames(df@meta.data) <- df@meta.data$ID 
    df@meta.data$ID <- NULL
    return (df)
}
healthy.df.filtered[[1]]<- metadata_change(healthy.df.filtered[[1]], 
                                            string = "_1")
healthy.df.filtered[[2]] <- metadata_change(healthy.df.filtered[[2]], 
                                            string = "_2")
healthy.df.filtered[[3]] <- metadata_change(healthy.df.filtered[[3]], 
                                            string = "_3")
moderate.df.filtered[[1]] <- metadata_change(moderate.df.filtered[[1]], 
                                            string = "_5")
moderate.df.filtered[[2]] <- metadata_change(moderate.df.filtered[[2]], 
                                            string = "_6")
moderate.df.filtered[[3]] <- metadata_change(moderate.df.filtered[[3]], 
                                            string = "_7")
severe.df.filtered[[1]] <- metadata_change(severe.df.filtered[[1]], 
                                            string = "_8")
severe.df.filtered[[2]] <- metadata_change(severe.df.filtered[[2]], 
                                            string = "_9")
severe.df.filtered[[3]] <- metadata_change(severe.df.filtered[[3]], 
                                            string = "_10")
severe.df.filtered[[4]] <- metadata_change(severe.df.filtered[[4]], 
                                            string = "_11")
severe.df.filtered[[5]] <- metadata_change(severe.df.filtered[[5]], 
                                            string = "_12")
severe.df.filtered[[6]] <- metadata_change(severe.df.filtered[[6]], 
                                            string = "_13")

In [18]:
all <- c(healthy.df.filtered, moderate.df.filtered, severe.df.filtered)

In [19]:
nCoV <- FindIntegrationAnchors(object.list = all, dims = 1:50)
nCoV.integrated <- IntegrateData(anchorset = nCoV, dims = 1:50,features.to.integrate = rownames(nCoV))
saveRDS(nCoV.integrated, file = "nCoV_mine.rds")

Warning message in CheckDuplicateCellNames(object.list = object.list):
“Some cell names are duplicated across objects provided. Renaming to enforce unique cell names.”
Computing 2000 integration features

Scaling features for provided objects

Finding all pairwise anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 22372 anchors

Filtering anchors

	Retained 3468 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 9406 anchors

Filtering anchors

	Retained 3365 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 9789 anchors

Filtering anchors

	Retained 3061 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 10785 anchors

Filtering anchors

	Retained 2581 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 11900 anchors

Filtering anchors

	Retained 2507 anchors

Running CCA

Merging objects

Finding neighborhoods



In [37]:
###first generate data and scale data in RNA assay
DefaultAssay(nCoV.integrated) <- "RNA"
nCoV.integrated[['percent.mito']] <- PercentageFeatureSet(nCoV.integrated, pattern = "^MT-")
nCoV.integrated <- NormalizeData(object = nCoV.integrated, normalization.method = "LogNormalize", scale.factor = 1e4)
nCoV.integrated <- FindVariableFeatures(object = nCoV.integrated, selection.method = "vst", nfeatures = 2000,verbose = FALSE)
nCoV.integrated <- ScaleData(nCoV.integrated, verbose = FALSE, vars.to.regress = c("nCount_RNA", "percent.mito"))

In [38]:
saveRDS(nCoV.integrated, file = "nCoV_mine_integrated_celltype.rds")

## Needed script 
```bash
conda install -c bioconda bioconductor-mast
```

In [39]:
##change to integrated assay
DefaultAssay(nCoV.integrated) <- "integrated"
dpi = 300
png(file="qc.png", width = dpi*16, height = dpi*8, units = "px",res = dpi,type='cairo')
VlnPlot(object = nCoV.integrated, features = c("nFeature_RNA", "nCount_RNA"), ncol = 2)
dev.off()

png(file="umi-gene.png", width = dpi*6, height = dpi*5, units = "px",res = dpi,type='cairo')
FeatureScatter(object = nCoV.integrated, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")
dev.off()

png 
  2

png 
  2

In [40]:
# Run the standard workflow for visualization and clustering
nCoV.integrated <- ScaleData(nCoV.integrated, verbose = FALSE, vars.to.regress = c("nCount_RNA", "percent.mito"))
nCoV.integrated <- RunPCA(nCoV.integrated, verbose = FALSE,npcs = 100)
nCoV.integrated <- ProjectDim(object = nCoV.integrated)
png(file="pca.png", width = dpi*10, height = dpi*6, units = "px",res = dpi,type='cairo')
ElbowPlot(object = nCoV.integrated,ndims = 100)
dev.off()

PC_ 1 
Positive:  VIM, SRGN, CTSL, CTSB, LGALS1, CCL2, VAMP5, CCL3, CD68, FTL 
	   PLEK, IFI30, TNFSF13B, CXCL10, BCL2A1, GRN, SNX10, MAFB, IL1RN, SERPINA1 
Negative:  TSPAN1, ELF3, SLC44A4, FXYD3, CD24, SMIM22, TMC5, CLDN4, KRT8, C20orf85 
	   RSPH1, C9orf24, C1orf194, CYP4B1, FAM183A, TACSTD2, MUC4, WFDC2, ZMYND10, CAPS 
PC_ 2 
Positive:  IL32, CD2, CD3E, GZMA, CD7, LCK, LIMD2, CCL5, CD3D, CD247 
	   SEPT1, TRBC2, SH2D2A, CD96, GZMB, CLEC2D, OCIAD2, PRF1, SPOCK2, CD3G 
Negative:  FTL, CTSB, GLUL, PSAP, CTSL, CTSD, CD68, GRN, IL1RN, SAT1 
	   CCL2, LGALS3, IFI30, SERPINA1, CD63, SOD2, NPC2, CSTB, MAFB, SGK1 
PC_ 3 
Positive:  S100A14, S100A16, KRT7, CEACAM6, F3, MAL2, C19orf33, MALL, PRSS23, ADIRF 
	   KLK10, PLS3, LMO7, AQP5, CXCL17, S100A2, KLK11, KRT19, BPIFB1, FHL2 
Negative:  C20orf85, SNTN, DNAH12, AC007906.2, C1orf194, TPPP3, CCDC78, TMEM190, LRRC46, C9orf116 
	   C9orf24, TEKT1, CAPSL, ZMYND10, LDLRAD1, FAM183A, PIFO, RSPH1, FAM92B, HYDIN 
PC_ 4 
Positive:  ISG15, S100A8, ISG2

png 
  2

In [41]:
###cluster
nCoV.integrated <- FindNeighbors(object = nCoV.integrated, dims = 1:50)
nCoV.integrated <- FindClusters(object = nCoV.integrated, resolution = 1.2) 

###tsne and umap
nCoV.integrated <- RunTSNE(object = nCoV.integrated, dims = 1:50)
nCoV.integrated <- RunUMAP(nCoV.integrated, reduction = "pca", dims = 1:50)
png(file="tsne.png", width = dpi*8, height = dpi*6, units = "px",res = dpi,type='cairo')
DimPlot(object = nCoV.integrated, reduction = 'tsne',label = TRUE)
dev.off()
png(file="umap.png", width = dpi*8, height = dpi*6, units = "px",res = dpi,type='cairo')
DimPlot(object = nCoV.integrated, reduction = 'umap',label = TRUE)
dev.off()

Computing nearest neighbor graph

Computing SNN



Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 63734
Number of edges: 2626093

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.8627
Number of communities: 31
Elapsed time: 23 seconds


1 singletons identified. 30 final clusters.

Warning message:
“The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric
To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
This message will be shown once per session”
15:56:42 UMAP embedding parameters a = 0.9922 b = 1.112

15:56:42 Read 63734 rows and found 50 numeric columns

15:56:42 Using Annoy for neighbor search, n_neighbors = 30

15:56:42 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

15:56:53 Writing NN index file to temp file /tmp/RtmpRyS3Mq/file37c56a94540d

15:56:53 Searching Annoy index using 1 thread, search_k = 3000

15:57:20 Annoy recall = 100%

15:57:21 Commencing smooth kNN distance calibra

png 
  2

png 
  2

In [42]:
DefaultAssay(nCoV.integrated) <- "RNA"
# find markers for every cluster compared to all remaining cells, report only the positive ones
nCoV.integrated@misc$markers <- FindAllMarkers(object = nCoV.integrated, assay = 'RNA',only.pos = TRUE, test.use = 'MAST')

write.table(nCoV.integrated@misc$markers,file='marker_MAST.txt',row.names = FALSE,quote = FALSE,sep = '\t')

dpi = 300
png(file="feature.png", width = dpi*24, height = dpi*5, units = "px",res = dpi,type='cairo')
VlnPlot(object = nCoV.integrated, features = c("nFeature_RNA", "nCount_RNA"))
dev.off()
saveRDS(nCoV.integrated, file = "nCoV_mine_integrated_celltype_final.rds")

Calculating cluster 0


 Completed [--------------------------------------------]   0% with 0 failures

 Completed [--------------------------------------------]   1% with 0 failures

 Completed [>-------------------------------------------]   1% with 0 failures

 Completed [>-------------------------------------------]   2% with 0 failures

 Completed [>-------------------------------------------]   3% with 0 failures

 Completed [=>------------------------------------------]   3% with 0 failures

 Completed [=>------------------------------------------]   4% with 0 failures

 Completed [=>------------------------------------------]   5% with 0 failures

 Completed [=>------------------------------------------]   6% with 0 failures

 Completed [==>-----------------------------------------]   6% with 0 failures

 Completed [==>-----------------------------------------]   7% with 0 failures

 Completed [==>-----------------------------------------]   8% with 0 failures

 Completed [===>

png 
  2

In [43]:
#Draw Heatmap
dpi <- 300
hc.markers = read.delim2("marker_MAST.txt",header = TRUE, stringsAsFactors = FALSE,check.names = FALSE, sep = "\t")
hc.markers %>% group_by(cluster) %>% top_n(n = 10, wt = avg_log2FC) -> top10
tt1 = DoHeatmap(object = subset(nCoV.integrated, downsample = 500), features = top10$gene) + NoLegend()
ggplot2::ggsave(file="marker_heatmap_MAST.pdf",plot = tt1,device = 'pdf',width = 20, height = 16, units = "in",dpi = dpi,limitsize = FALSE)

Warning message in DoHeatmap(object = subset(nCoV.integrated, downsample = 500), :
“The following features were omitted as they were not found in the scale.data slot for the RNA assay: BCL11A, HERPUD1, TCF4, MEF2C, SMIM14, FTL, TAOK1, PRDX1, ATOX1, GLRX, ALDOA, PSMA6, PPA1, IFITM2, SNHG25, RPL31, H2AFZ, RPL13A, RPL36A, RPS10, RPL21, RPS20, RPL17, HSP90AA1, CTSZ, MT-ND2, MT-ND3, MT-ND4, MT-ATP6, MT-CO3, LYZ, HLA-DPB1, RPS17, CD52, CSTA, ALOX5AP, CRIP1, DUSP6, VAMP5, MARCKS, DEFB1”


## After integrating and reducing the dimensionality of the data (clustering) are complete 

In [44]:
markers = c('AGER','SFTPC','SCGB3A2','TPPP3','KRT5',
            'CD68','FCN1','CD1C','TPSB2','CD14','MARCO','CXCR2',
            'CLEC9A','IL3RA',
            'CD3D','CD8A','KLRF1',
            'CD79A','IGHG4','MS4A1',
            'VWF','DCN',
            'FCGR3A','TREM2','KRT18')
hc.markers = read.delim2("marker_MAST.txt",header = TRUE, stringsAsFactors = FALSE,check.names = FALSE, sep = "\t")
hc.markers %>% group_by(cluster) %>% top_n(n = 30, wt = avg_log2FC) -> top30
var.genes = c(nCoV.integrated@assays$RNA@var.features,top30$gene,markers)
nCoV.integrated <- ScaleData(nCoV.integrated, verbose = FALSE, vars.to.regress = c("nCount_RNA", "percent.mito"),features = var.genes)
saveRDS(nCoV.integrated, file = "nCoV_mine_integrated_celltype_final_marker.rds")

In [45]:
dpi <- 300
png(file="tsne_markers.png", width = dpi*8, height = dpi*6, units = "px",res = dpi,type='cairo')
DimPlot(object = nCoV.integrated, reduction = 'tsne',label = TRUE)
dev.off()
png(file="umap_markers.png", width = dpi*8, height = dpi*6, units = "px",res = dpi,type='cairo')
DimPlot(object = nCoV.integrated, reduction = 'umap',label = TRUE)
dev.off()

png 
  2

png 
  2

In [125]:
####marker expression
dpi = 300
markers = c('AGER','SFTPC','SCGB3A2','TPPP3','KRT5',
            'CD68','FCN1','CD1C','TPSB2','CD14','MARCO','CXCR2',
            'CLEC9A','IL3RA',
            'CD3D','CD8A','KLRF1',
            'CD79A','IGHG4','MS4A1',
            'VWF',
            'FCGR3A','TREM2','KRT18','HBB')
#markers = c('HBB')
#markers = c('DCN') --> No DCN
png(file="violin_marker.png", width = dpi*30, height = dpi*24, units = "px",res = dpi,type='cairo')
print(VlnPlot(object = nCoV.integrated, features = markers,pt.size = 0)+ theme(axis.text.x = element_text(angle = 90,hjust = 1,vjust = 0.5)))
dev.off()
png(file="umap_marker.png", width = dpi*30, height = dpi*36, units = "px",res = dpi,type='cairo')
print(FeaturePlot(object = nCoV.integrated, features = markers,cols = c("lightgrey","#ff0000")))
dev.off()
for(marker in markers){
  png(file=paste("violin_",marker,".png",sep=''), width = dpi*8, height = dpi*3, units = "px",res = dpi,type='cairo')
  print(VlnPlot(object = nCoV.integrated, features = marker,pt.size = 0)+ theme(axis.text.x = element_text(angle = 90,hjust = 1,vjust = 0.5)))
  dev.off()
  png(file=paste("umap_",marker,".png",sep=''), width = dpi*6, height = dpi*4, units = "px",res = dpi,type='cairo')
  print(FeaturePlot(object = nCoV.integrated, features = marker,cols = c("lightgrey","#ff0000")))
  dev.off()
}

png 
  2

png 
  2

In [127]:
markers = c('AGER','SFTPC','SCGB3A2','TPPP3','KRT5',
            'CD68','FCN1','CD1C','TPSB2','CD14','MARCO','CXCR2',
            'CLEC9A','IL3RA',
            'CD3D','CD8A','KLRF1',
            'CD79A','IGHG4','MS4A1',
            'VWF',
            'FCGR3A','TREM2','KRT18','HBB')

pdf(file="marker_heatmap.pdf", width = 10, height = 8)
pp = DotPlot(nCoV.integrated, features = rev(markers),cols = c('white','#F8766D'),dot.scale =5) + RotatedAxis()
pp = pp + theme(axis.text.x = element_text(size = 12),axis.text.y = element_text(size = 12)) + labs(x='',y='') + 
  guides(color = guide_colorbar(title = 'Scale expression'),size = guide_legend(title = 'Percent expressed')) + 
  theme(axis.line = element_line(size = 0.6))
print(pp)
dev.off()

png 
  2